# Introducción 

La idea de la práctica es visitar aquellos temas que en cierta manera nos permitan ver más contenido del curso.

La práctica esta dividida en 4 o 5 subapartados, que ya tenéis en este mismo Notebook. Estos subapartados estan aquí para que rellenéis el código que hace falta para la realización de la práctica. Obviamente podéis usar tantas celdas como os hagan falta, es más es de agradecer si el código final esta algo "limpio". Usar funciones, algo de comentario, etc, etc...

Usaremos 2 datasets, uno para el primer ejercicio, y otro para el resto de ejercicios.

Ejercicios:


1.   Machine Learning vs Deep Learning (Acordaros que hay que implementar el pipeline visto en clase entero)

    1.1. Implementación de un modelo de Sentiment Analysis con algún algoritmo de Machine Learning Clásico.
    
    1.2. Implementación de un modelo de Sentiment Analysis con alguna arquitectura de Deep Learning.
    
    1.3. Breve Comparación de resultados. Confusion Matrix.
    
2. Hacer Analysis de los tweets del segundo dataset. Que temas aparecen? Como se representan estos temas? De que hablan unos y otros?

3. Escoged a uno de los dos presidentes, y escribid tweets como ellos, usando un Modelo Generativo.

En cada ejercicio, espero explicaciones y razonamientos del porque una arquitectura y no otra, por ejemplo en Deep Learning, porque usar Convolutionals en lugar de recurrentes, o en Machine Learning, Bayes en lugar de SVM. Hay que explicar el pipeline, sobretodo el preproceso de datos, con lo que habrá que hacer un pequeño estudio de que datos tenemos, y si hay cosas que se pueden ignorar, si hacéis stemming, o no, etc, etc...

Acordaros de que objetivo final no es que obtengáis una accuracy brutal, es que comprendais que pasa cuando usais un algoritmo u otro, y que problemas o beneficios nos dan.

![](https://i.pinimg.com/736x/19/63/8c/19638c0b33e2f7822d6806ce31d89d84--funny-cartoons-funny-jokes.jpg)

Mucha suerte y ánimo!



## Librerias

In [0]:
!pip install gensim
!pip install pyLDAvis
#!pip install stop_words

# Spacy
Como vasmo a trabajar con datasets en ingles usaremos el modelo **"en_core_web_md"**. Este modelo no viene precargado con la version de spacy de colab por lo que deberemos instalarlo nosotros

In [0]:
!python -m spacy download en_core_web_md  # ocupa 95.4 MB

     |████████████████████████████████| 95.4MB 1.2MB/s 
  Created wheel for en-core-web-md: filename=en_core_web_md-2.1.0-cp36-none-any.whl size=97126236 sha256=7ebf1def4b28fd117268c8790e2a40cb964b6e65597feaf6957f85361ee1bc21
  Stored in directory: /tmp/pip-ephem-wheel-cache-ngvkon0l/wheels/c1/2c/5f/fd7f3ec336bf97b0809c86264d2831c5dfb00fc2e239d1bb01
Successfully built en-core-web-md
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_md')


In [0]:
#Hay dos maneras de cargar el modelos "instalado" arriba:

#Modo 1:
# Se necesita reiniciar el entorno de ejecucion
# Pero podemos meter opciones en la carga
import spacy
nlp = spacy.load('en_core_web_md', disable= ['ner', 'parser'])

In [0]:
#Modo 2:
# De esta otra forma es necesario hacer el import, pero no necesitamos reiniciar el entorno de ejecucion
import en_core_web_md
nlp = en_core_web_md.load()

# 1.1. Implementación de un modelo de Sentiment Analysis con algún algoritmo de Machine Learning Clásico

In [0]:
%matplotlib inline

import numpy as np
import pandas as pd
import spacy
import io

from time import time

import pickle
import json
import os
import csv

import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
import pandas as pd

from random import sample

import warnings
warnings.filterwarnings('ignore')

from sklearn.model_selection import StratifiedShuffleSplit

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB, BernoulliNB
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline



## Funciones

In [0]:
def split_train_test(dataset, split=0.2):

    x, y = zip(*dataset)
    x = np.array(list(x))
    y = np.array(list(y))
    sss = StratifiedShuffleSplit(n_splits=1, test_size=split, random_state=1337) #l33t seed
    for train_index, test_index in sss.split(x, y):
        x_train, x_test = x[train_index], x[test_index]
        y_train, y_test = y[train_index], y[test_index]
    splits = {'train':(x_train, y_train), 'test':(x_test, y_test)}
    return splits

# En la funcion preprocess pondremos diferentes opciones de preprocesado del texto
# e iremos probando con distintas combinaciones (comentando y descomentando) para ver
# cual nos genera un mejor modelo.
def preprocess(text, lema = True):

  # Eliminamos los espacios en blanco por delante y detras
  text = text.strip()

  #Tokenizamos
  doc = nlp(text)
  words = [t for t in doc]

  # Quitamos los signos de puntuacion
  #words = [t for t in doc if not t.is_punct]
  
  # Quitamos las stop_words
  #words = [t for t in words if not t.is_stop]

  # Eliminamos palabras de tamaño menor de tres
  #words = [t for t in words if len(t) > 2 and  t.text.isalpha()]
  
  # Lematizamos
  if(lema):
    words = [t.lemma_ for t in words]
  else:
    words = [t.text for t in words]
  
  # Ponemos las palabras en minusculas
  words = [t.lower() for t in words]

  # Quitamos las direcciones de los usuarios
  #words = list(filter(lambda x:x[0]!='@', words))  

  # Por ultimo eliminamos los caracteres que no queremos que aparezcan
  all_printables_chars = '0123456789abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~ \t\n\r\x0b\x0c'
  printables_chars = '0123456789abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ!"\' ?-'
  #words = [''.join([c if c in printables_chars else '' for c in word]) for word in words]
  
  # Generamos el string de salida
  result = ' '.join(words)
  return result.strip()

def get_dataset(df, prep=True, lema=True):

  sentiment_dataset = []
  for index, row in df.iterrows():
    if(index%500==0):
      print(f' {index} elements preprocessed')
    label = row[1] 
    sentence = row[2]
    # Aqui ira el preprocesado de texto
    if(prep):
      sentence =  preprocess(sentence,lema)
    sentiment_dataset.append((sentence, label))
  return sentiment_dataset



## Dataset

Para el ejercicio 1.1 cargaremos el dataset train_sentiment.csv

In [0]:
from google.colab import files
uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))


In [0]:
df = pd.read_csv(io.StringIO(uploaded['train_sentiment.csv'].decode('ISO-8859-1')))
df

### Sample Dataset
El dataset train_sentiment contiene 99989 registros, lo cual hace que los tiempos tanto de preproceso como de entrenamiento sean muy largos (mas de 2 horas) e inviables para poder hacer pruebas.
**Por tanto el como el objetivo de esta practica no es conseguir Scores excelentes sino plasmar lo aprendido en el modulo, usaremos un sample de 5.000 registros**

In [0]:
#Establecemos un dataset mas pequeño para hacer pruebas
df_smaple = df[0:5000]

## Prepocesado
Vamos a jugar con distintas opciones: stop_words, string.printable,lematizacion.... y veremos con cual obtenemos un mejor accuracy

In [0]:
# En primer lugar generamos el datset como una lista de tuplas (frase, etiqueta)
# Las etiquetas son 0 (sentimiento negativo) y 1 (sentimiento positivo), las vamos a dejar asi, porque no vale
# la pena ponerle etiquetas de texto como "POS" y "NEG"
# El preprocesado de datos (funcion preprocess) se lleva a cabo dentro de la propia funcion que genera el dataset (get_dataset), 
# el segundo parametro corresponde a si se lleva a cabo el preproceso y el tercero a si se hace la lemmatizacion
start_time = time()
sentiment_dataset = get_dataset(df_smaple, True, True)
elapsed_time = time() - start_time
print("Preprocess Elapsed time: %0.10f seconds." % elapsed_time)
print(sentiment_dataset[0])
len(sentiment_dataset)


 0 elements preprocessed
 500 elements preprocessed
 1000 elements preprocessed
 1500 elements preprocessed
 2000 elements preprocessed
 2500 elements preprocessed
 3000 elements preprocessed
 3500 elements preprocessed
 4000 elements preprocessed
 4500 elements preprocessed
Preprocess Elapsed time: 14.1419386864 seconds.
('be so sad for -pron- apl friend .............', 0)


5000

## Split Train/Test

In [0]:
split = split_train_test(sentiment_dataset, 0.20)


## Pipeline Bayes (BernoulliNB)
**Usamos Bernoulli en lugar de Multinomial porque la clasificacion es Binomial**

In [0]:
pipeline_bayes = Pipeline([
    ('vect', CountVectorizer(max_df=0.5)),
    #('tfidf', TfidfTransformer()), #No vale la pena usar esta normalizacion porque no se consiguen mejores resultados
    ('clf', BernoulliNB()),])
pipeline_bayes.get_params().keys() # que parametros podemos tocar en el gridsearch!

dict_keys(['memory', 'steps', 'verbose', 'vect', 'clf', 'vect__analyzer', 'vect__binary', 'vect__decode_error', 'vect__dtype', 'vect__encoding', 'vect__input', 'vect__lowercase', 'vect__max_df', 'vect__max_features', 'vect__min_df', 'vect__ngram_range', 'vect__preprocessor', 'vect__stop_words', 'vect__strip_accents', 'vect__token_pattern', 'vect__tokenizer', 'vect__vocabulary', 'clf__alpha', 'clf__binarize', 'clf__class_prior', 'clf__fit_prior'])

### GridsearchCV and parameters Bayes

In [0]:
param_grid_bayes = {
    'vect__max_df': (0.5, 0.75, 1.0),
    'vect__ngram_range': ((1, 2), (1, 3), (2, 3)),  #ngrams to test
    'vect__analyzer': ('word', 'char', 'char_wb'),
    #'tfidf__use_idf': (True, False),
    #'tfidf__norm': ('l1', 'l2'),
    'clf__alpha': (0.1, 1.0)    
}

grid_search_bayes = GridSearchCV(pipeline_bayes, param_grid_bayes,cv=5, n_jobs=-1, verbose=1)

In [0]:
grid_search_bayes.fit(split['train'][0], split['train'][1])
best_parameters = grid_search_bayes.best_estimator_.get_params()
for param_name in sorted(param_grid_bayes.keys()):
    print("\t%s: %r" % (param_name, best_parameters[param_name]))
print("Best score: %0.3f" % grid_search_bayes.best_score_)

Fitting 5 folds for each of 54 candidates, totalling 270 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:    7.2s
[Parallel(n_jobs=-1)]: Done 196 tasks      | elapsed:   44.0s


	clf__alpha: 0.1
	vect__analyzer: 'word'
	vect__max_df: 0.75
	vect__ngram_range: (1, 2)
Best score: 0.770


[Parallel(n_jobs=-1)]: Done 270 out of 270 | elapsed:  1.1min finished


In [0]:
pipeline_bayes.set_params(**best_parameters)
predictions = pipeline_bayes.score(split['test'][0], split['test'][1])
print('TEST SCORE BAYES: {}'.format(predictions))

TEST SCORE BAYES: 0.77


## Pipeline SVM (SVC)

In [0]:
# Para el caso de SVC hemos comprobado que los tiempos de entrenamiento para este algoritmo son muy elevados
# Por ello, usaremos los parametros ya calculados para Benoulli y solo haremos tunning con los propios de SVC 
pipeline_svc = Pipeline([
    ('vect', CountVectorizer(max_df=0.75,ngram_range=(1,2),analyzer='word')),
    #('tfidf', TfidfTransformer(norm='l1',use_idf=True)), #Sin esta normalizacion obtenemos mejores resultados
    ('clf', SVC()),])
pipeline_svc.get_params().keys() # que parametros podemos tocar en el gridsearch!

dict_keys(['memory', 'steps', 'verbose', 'vect', 'clf', 'vect__analyzer', 'vect__binary', 'vect__decode_error', 'vect__dtype', 'vect__encoding', 'vect__input', 'vect__lowercase', 'vect__max_df', 'vect__max_features', 'vect__min_df', 'vect__ngram_range', 'vect__preprocessor', 'vect__stop_words', 'vect__strip_accents', 'vect__token_pattern', 'vect__tokenizer', 'vect__vocabulary', 'clf__C', 'clf__break_ties', 'clf__cache_size', 'clf__class_weight', 'clf__coef0', 'clf__decision_function_shape', 'clf__degree', 'clf__gamma', 'clf__kernel', 'clf__max_iter', 'clf__probability', 'clf__random_state', 'clf__shrinking', 'clf__tol', 'clf__verbose'])

### GridsearchCV and parameters SVC

In [0]:
# Para el caso de SVC
# Hemos comprobado que los tiempos de entrenamiento para este algoritmo son prohibitivos
# por lo que usaremos los parametros ya calculados para Benoully (que son los valores por defecto para max_df y analyzer y por eso
# los quitamos y para el caso del ngram_range es (1,3))
param_grid_svc = {
    #'vect__max_df': (0.5, 0.75, 1.0),
    #'vect__ngram_range': ((1, 2), (1, 3), (2, 3)),  #ngrams to test
    #'vect__analyzer': ('word', 'char', 'char_wb'),
    #'tfidf__use_idf': (True, False),
    # 'tfidf__norm': ('l1', 'l2'),
    'clf__C': (0.1, 1.0),
    'clf__gamma':(0.001, 0.0001)
}
grid_search_svc = GridSearchCV(pipeline_svc, param_grid_svc,cv=5, n_jobs=-1, verbose=1)

In [0]:
grid_search_svc.fit(split['train'][0], split['train'][1])
best_parameters = grid_search_svc.best_estimator_.get_params()
for param_name in sorted(param_grid_svc.keys()):
    print("\t%s: %r" % (param_name, best_parameters[param_name]))
print("Best score: %0.3f" % grid_search_svc.best_score_)

Fitting 5 folds for each of 4 candidates, totalling 20 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  20 out of  20 | elapsed:   23.0s finished


	clf__C: 0.1
	clf__gamma: 0.001
Best score: 0.711


In [0]:
pipeline_svc.set_params(**best_parameters)
predictions = pipeline_svc.score(split['test'][0], split['test'][1])
print('TEST SCORE SVC: {}'.format(predictions))

TEST SCORE SVC: 0.711


## Resumen
Hemos llevado a cabo numerosas combinaciones con el procesado y finalmente el que mejor funciona es la siguiente combinacion:

1.   Hacer strip() para eliminar espacios en blanco al principio y al final
2.  Lemmatizar
3.  Pasar palabras a minusculas
4.  Quitar el nombre de los usuarios (eliminar palabras que empiecen por "@")
5.  Eliminar caracteres que no nos interesan


En cuanto a los modelos aplicados, hemos probado dos con los siguientes Scores:

*   BernoulliNB --> TEST SCORE: 0.77
*   SVC --> TEST SCORE: 0.711







# 1.2. Implementación de un modelo de Sentiment Analysis con alguna arquitectura de Deep Learning.

Usaremos funciones e imports del apartado anterior
Implementaremos los tres modelos que vimos en clase: DAN, CNN y LSTM

## Dataset/Vocabulary/Input
Usaremos el mismo dataset que en el punto 1.1 (sentiment_dataset)

In [0]:
# El primer paso sera generar el vocabularios y el tamaño del input
from collections import Counter
tokens = []
tokenized = []
for x, y in sentiment_dataset:
    x_t = nlp(x)
    toks = [t.text for t in x_t]
    tokens+= toks
    tokenized.append((toks, y))
    
vocab_counter = Counter(tokens)
vocab = set(tokens)
print('Num de features a usar: ', len(vocab))  
print(len(tokenized))

Num de features a usar:  8855
5000


In [0]:
maxlen = max([len(x) for x, _ in tokenized])
maxlen

89

In [0]:
lens = [len(x) for x, _ in tokenized]
median = np.median(np.array(lens))
mean = np.mean(np.array(lens))
maxlen = int(median)*2
print(median, mean, maxlen)

13.0 14.5162 26


In [0]:
tokenized_filtered = [(x, y) for x, y in tokenized if len(x) < maxlen]
len(tokenized_filtered)

4263

En este caso vemos que usando ese valor de maxlen estamos eliminando casi el 15% de los registros lo cual es demasiado elevado por tanto cogeremos 40 como maxlen

In [0]:
maxlen = 40
tokenized_filtered = [(x, y) for x, y in tokenized if len(x) < maxlen]
len(tokenized_filtered)


4989

In [0]:
# Establecemos los vectores de conversion
labels = ['0','1']
w2id = {k:i for i, k in enumerate(vocab)}
w2id['<UNK>'] = len(w2id)
l2id = {label:i for i, label in enumerate(labels)}

In [0]:
# Preparar Input. Padding. Conversión a input
input_ready = []
for x, y in tokenized_filtered:
    sentence = np.zeros((maxlen))
    label = np.zeros((len(labels)))
    label[int(y)] = 1
    for i, t in enumerate(x):
        sentence[i] = w2id[t] if t in vocab_counter and vocab_counter[t]>=5 else w2id['<UNK>']
    input_ready.append((sentence,label))

In [0]:
input_ready[0][0].shape

(40,)

## Split

In [0]:
splits = split_train_test(input_ready)
print(splits['train'][0].shape)
print(splits['train'][1].shape)
splits['train'][1]

(3991, 40)
(3991, 2)


array([[1., 0.],
       [0., 1.],
       [1., 0.],
       ...,
       [1., 0.],
       [1., 0.],
       [0., 1.]])

## DAN (Deep Averaging Networks)

In [0]:
# Tipos de Capas que usaremos como Lego Bricks
import numpy as np
from keras.layers import Conv1D, Conv2D, SimpleRNN, LSTM, Dense, Dropout
from keras.models import Sequential, Model
from keras.layers import Input, Average, average, Lambda, Embedding, Flatten


from keras import backend as K

In [0]:
def mask_aware_mean(x):
    # recreate the masks - all zero rows have been masked
    mask = K.not_equal(K.sum(K.abs(x), axis=2, keepdims=True), 0)
    # number of that rows are not all zeros
    n = K.sum(K.cast(mask, 'float32'), axis=1, keepdims=False)
    # compute mask-aware mean of x
    x_mean = K.sum(x, axis=1, keepdims=False) / n
    return x_mean

def mask_aware_mean_output_shape(input_shape):
    shape = list(input_shape)
    assert len(shape) == 3
    return (shape[0], shape[2])

In [0]:
input_layer = Input(shape=(maxlen,) )
embedding = Embedding(output_dim=100, input_dim=len(w2id), input_length=maxlen)(input_layer)
doc_representation = Lambda(mask_aware_mean, mask_aware_mean_output_shape, name='embedding_average')(embedding)
dense_1 = Dense(100, activation='relu')(doc_representation)
drop_1 = Dropout(0.5)(dense_1)
dense_2 = Dense(100, activation='relu')(drop_1)
drop_2 = Dropout(0.5)(dense_2)
out = Dense(len(l2id), activation='softmax')(drop_2)

model_DAN = Model(inputs=input_layer, outputs=out)

model_DAN.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

model_DAN.summary()

Model: "model_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_11 (InputLayer)        (None, 40)                0         
_________________________________________________________________
embedding_10 (Embedding)     (None, 40, 100)           885600    
_________________________________________________________________
embedding_average (Lambda)   (None, 100)               0         
_________________________________________________________________
dense_25 (Dense)             (None, 100)               10100     
_________________________________________________________________
dropout_17 (Dropout)         (None, 100)               0         
_________________________________________________________________
dense_26 (Dense)             (None, 100)               10100     
_________________________________________________________________
dropout_18 (Dropout)         (None, 100)               0   

### Model Fit

In [0]:
model_DAN.fit(splits['train'][0], splits['train'][1],
          epochs=50,
          batch_size=12)

Epoch 1/50
3991/3991 [==============================] - 2s 490us/step - loss: 0.5878 - acc: 0.7108
Epoch 2/50
3991/3991 [==============================] - 1s 345us/step - loss: 0.5178 - acc: 0.7419
Epoch 3/50
3991/3991 [==============================] - 2s 380us/step - loss: 0.4683 - acc: 0.7840
Epoch 4/50
3991/3991 [==============================] - 1s 371us/step - loss: 0.4415 - acc: 0.8056
Epoch 5/50
3991/3991 [==============================] - 1s 346us/step - loss: 0.4209 - acc: 0.8116
Epoch 6/50
3991/3991 [==============================] - 1s 355us/step - loss: 0.4029 - acc: 0.8256
Epoch 7/50
3991/3991 [==============================] - 2s 376us/step - loss: 0.3911 - acc: 0.8224
Epoch 8/50
3991/3991 [==============================] - 1s 352us/step - loss: 0.3857 - acc: 0.8306
Epoch 9/50
3991/3991 [==============================] - 1s 351us/step - loss: 0.3793 - acc: 0.8336
Epoch 10/50
3991/3991 [==============================] - 1s 357us/step - loss: 0.3710 - acc: 0.8376
Epoch 11/

### Model Test

In [0]:
score_avg_emb = model_DAN.evaluate(splits['test'][0], splits['test'][1])
score_avg_emb

998/998 [==============================] - 0s 188us/step


[1.455417138541151, 0.7635270541082164]

### Conclusion
A la vista de la diferencia entre el score del train y del test podemos decir que tenemos un alto Overfiting, el cual se produce practicamente desde el comienzo del entreno (epoca 3).

Podemos observar que el valor de test se acerca mucho a los valores que obtuvimos con Bayes.

## CNN

In [0]:
from keras.layers import MaxPooling1D, GlobalMaxPooling2D
from keras.layers import Input, Embedding, Concatenate, Reshape
from keras.models import Model

In [0]:
embedding_dim = 100

input_layer = Input(shape=(maxlen,) )# maxlen
embedding = Embedding(output_dim=embedding_dim, input_dim=len(w2id), input_length=maxlen)(input_layer)#w2id maxlen
reshape = Reshape((maxlen,embedding_dim,1))(embedding)

conv_1 = Conv2D(filters=50, kernel_size=(1, embedding_dim), activation='relu', padding='valid')(reshape)
mp_1 = GlobalMaxPooling2D()(conv_1)

conv_2 = Conv2D(filters=50, kernel_size=(2, embedding_dim), activation='relu', padding='valid')(reshape)
mp_2 = GlobalMaxPooling2D()(conv_2)

conv_5 = Conv2D(filters=50, kernel_size=(5, embedding_dim), activation='relu', padding='valid')(reshape)
mp_5 = GlobalMaxPooling2D()(conv_5)

doc_representation = Concatenate()([mp_1, mp_2, mp_5])

dense_1 = Dense(100, activation='relu')(doc_representation)
drop_1 = Dropout(0.5)(dense_1)
dense_2 = Dense(100, activation='relu')(drop_1)
drop_2 = Dropout(0.5)(dense_2)
out = Dense(len(l2id), activation='softmax')(drop_2)

model_CNN = Model(inputs=input_layer, outputs=out)

model_CNN.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

model_CNN.summary()

Model: "model_12"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_14 (InputLayer)           (None, 40)           0                                            
__________________________________________________________________________________________________
embedding_13 (Embedding)        (None, 40, 100)      885600      input_14[0][0]                   
__________________________________________________________________________________________________
reshape_3 (Reshape)             (None, 40, 100, 1)   0           embedding_13[0][0]               
__________________________________________________________________________________________________
conv2d_7 (Conv2D)               (None, 40, 1, 50)    5050        reshape_3[0][0]                  
___________________________________________________________________________________________

### Model Fit

In [0]:
model_CNN.fit(splits['train'][0], splits['train'][1],
          epochs=5,
          batch_size=12)

Epoch 1/5
3991/3991 [==============================] - 3s 834us/step - loss: 0.5706 - acc: 0.7161
Epoch 2/5
3991/3991 [==============================] - 2s 492us/step - loss: 0.4677 - acc: 0.7820
Epoch 3/5
3991/3991 [==============================] - 2s 532us/step - loss: 0.4033 - acc: 0.8251
Epoch 4/5
3991/3991 [==============================] - 2s 543us/step - loss: 0.3328 - acc: 0.8579
Epoch 5/5
3991/3991 [==============================] - 2s 558us/step - loss: 0.2705 - acc: 0.8940


### Model Test

In [0]:
score_cnn = model_CNN.evaluate(splits['test'][0], splits['test'][1], batch_size=16)
score_cnn

998/998 [==============================] - 0s 428us/step


[0.5989158143380839, 0.7785571141090087]

### Conclusion
En este caso el overfiting es aun mayor y tenemos que pararlo pronto ya que a prtir de la epoca 10 ya estamos en un escore del 98% para train.

LSTM

In [0]:
from keras.layers import CuDNNLSTM

In [0]:
rnn_type = CuDNNLSTM(100)

In [0]:
embedding_dim = 100

input_layer = Input(shape=(maxlen,) )
embedding = Embedding(output_dim=embedding_dim, input_dim=len(w2id), input_length=maxlen)(input_layer)
rnn = rnn_type(embedding)
dense_1 = Dense(100, activation='relu')(rnn)
drop_1 = Dropout(0.5)(dense_1)
dense_2 = Dense(100, activation='relu')(drop_1)
drop_2 = Dropout(0.5)(dense_2)
out = Dense(len(l2id), activation='softmax')(drop_2)

model_LSTM = Model(inputs=input_layer, outputs=out)

model_LSTM.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

model_LSTM.summary()


Model: "model_14"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_16 (InputLayer)        (None, 40)                0         
_________________________________________________________________
embedding_15 (Embedding)     (None, 40, 100)           885600    
_________________________________________________________________
cu_dnnlstm_1 (CuDNNLSTM)     (None, 100)               80800     
_________________________________________________________________
dense_40 (Dense)             (None, 100)               10100     
_________________________________________________________________
dropout_27 (Dropout)         (None, 100)               0         
_________________________________________________________________
dense_41 (Dense)             (None, 100)               10100     
_________________________________________________________________
dropout_28 (Dropout)         (None, 100)               0  

### Model Fit

In [0]:
model_LSTM.fit(splits['train'][0], splits['train'][1],
          epochs=5,
          batch_size=12)

Epoch 1/5
3991/3991 [==============================] - 2s 613us/step - loss: 0.3889 - acc: 0.8331
Epoch 2/5
3991/3991 [==============================] - 2s 603us/step - loss: 0.3818 - acc: 0.8449
Epoch 3/5
3991/3991 [==============================] - 3s 663us/step - loss: 0.3776 - acc: 0.8436
Epoch 4/5
3991/3991 [==============================] - 2s 612us/step - loss: 0.3644 - acc: 0.8514
Epoch 5/5
3991/3991 [==============================] - 2s 614us/step - loss: 0.3592 - acc: 0.8539


### Model Test

In [0]:
score_lstm = model_LSTM.evaluate(splits['test'][0], splits['test'][1], batch_size=12)
score_lstm


998/998 [==============================] - 0s 242us/step


[0.7566567698795953, 0.8086172343494897]

### Conclusion
El modelo LSTM es el que mejor resultado nos aporta con un score que por primera vez supera el 80%

# 1.3. Breve Comparación de resultados. Confusion Matrix.
De momento no se como hacer una matriz de confusion.

In [0]:
split_prediction = split_train_test(sentiment_dataset, 0.20)
x_sample=split_prediction['test'][0][:50]
y_sample=split_prediction['test'][1][:50]

In [0]:
pipeline_bayes.classes_

array([0, 1])

In [0]:
predictions_probs_bayes = pipeline_bayes.predict_proba(x_sample)
predictions_probs_bayes

array([[9.99474147e-01, 5.25853444e-04],
       [9.99939587e-01, 6.04128175e-05],
       [9.51427731e-01, 4.85722691e-02],
       [9.06615999e-01, 9.33840010e-02],
       [8.06762562e-01, 1.93237438e-01],
       [9.99279479e-01, 7.20521414e-04],
       [9.98494262e-01, 1.50573843e-03],
       [9.99999769e-01, 2.31110422e-07],
       [9.99909354e-01, 9.06462067e-05],
       [9.99860560e-01, 1.39439944e-04],
       [9.99981696e-01, 1.83044817e-05],
       [5.60618127e-01, 4.39381873e-01],
       [8.88443609e-01, 1.11556391e-01],
       [9.99994472e-01, 5.52809424e-06],
       [9.87665982e-01, 1.23340175e-02],
       [9.28280035e-01, 7.17199649e-02],
       [9.99750979e-01, 2.49021377e-04],
       [9.91863918e-01, 8.13608219e-03],
       [1.96562740e-01, 8.03437260e-01],
       [9.86934704e-01, 1.30652956e-02],
       [6.65833499e-01, 3.34166501e-01],
       [9.99999972e-01, 2.78298484e-08],
       [1.00000000e+00, 8.40909403e-12],
       [9.51970782e-01, 4.80292179e-02],
       [9.657203